# Weather Trader Version 2
This version is testing the trader on different sectors via etfs for those sectors

In [1]:
import vectorbt as vbt
import numpy as np
import pandas as pd
import nbformat
import datetime as dt
import scipy.stats as stats
import plotly.graph_objects as go

In [2]:
# securities = vbt.YFData.download(symbols=['SPY', 'XLK', 'XLV', 'XLF', 'XLP', 'XLI', 'XLE', 'XLU', 'XLRE', 'XLY', 'XLB'])
securities = vbt.YFData.download(symbols=['SPY', 'BND'])
sectors = securities.data.keys()
df = pd.DataFrame(securities.data['SPY']['Close'])

/Users/johnboesen/Documents/Code/#Algotraders/yja/yja-env/lib/python3.9/site-packages/vectorbt/data/base.py:527: UserWarning: Symbols have mismatching index. Setting missing data points to NaN.
  data = cls.align_index(data, missing=missing_index)


In [3]:
# securities.data['XLK'].dropna()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
1998-12-22 05:00:00+00:00,24.673167,24.744546,24.197310,24.399549,300500.0,0.0,0.0
1998-12-23 05:00:00+00:00,24.934886,24.982471,24.482822,24.982471,338300.0,0.0,0.0
1998-12-24 05:00:00+00:00,24.982464,24.982464,24.792122,24.887293,243700.0,0.0,0.0
1998-12-28 05:00:00+00:00,25.267978,25.267978,24.815915,24.958672,309300.0,0.0,0.0
1998-12-29 05:00:00+00:00,25.077644,25.137126,24.744544,25.030058,123800.0,0.0,0.0
...,...,...,...,...,...,...,...
2022-12-20 05:00:00+00:00,124.760002,126.250000,124.160004,125.610001,6311000.0,0.0,0.0
2022-12-21 05:00:00+00:00,126.190002,128.289993,125.879997,127.690002,7335300.0,0.0,0.0
2022-12-22 05:00:00+00:00,126.089996,126.230003,122.360001,124.470001,7661800.0,0.0,0.0


In [3]:
# create a new column for each sector
df = pd.DataFrame()
for sector in sectors:
    df[sector] = securities.data[sector]['Close']
# to csv
# df.to_csv('sectors.csv')

In [4]:
# drop null
df = df.dropna()
df.head()

,SPY,BND
Date,,
2007-04-10 04:00:00+00:00,106.189384,46.617035
2007-04-11 04:00:00+00:00,105.756165,46.493122
2007-04-12 04:00:00+00:00,106.226074,46.486931
2007-04-13 04:00:00+00:00,106.710747,46.412579
2007-04-16 04:00:00+00:00,107.724068,46.455948


In [6]:
# export to csv
# df.to_csv('sectors_no_nan.csv')

In [7]:
# df.index = pd.Series(df.index).dt.tz_localize(None)
# # add each sector to df
# for sector in sectors:
#     df[sector] = securities.data[sector]['Close']
# # drop rows with nan

In [8]:
# df.head()
# # make a copy of df
# df2 = df.copy()
# df2.dropna(how='any', inplace=True)
# df2.head()

In [9]:

# df = df.dropna()
# df.head()
# # export to csv
# df.to_csv('sectors.csv')

In [10]:
# df = pd.read_csv('sectors.csv', index_col=0)

In [5]:
# import weather_hist into a dataframe
weather_hist = pd.read_csv('data/weather_hist.csv', index_col=0) 
weather_hist.index = pd.to_datetime(weather_hist.index)

In [6]:
df.index = pd.to_datetime(df.index, utc = True)

In [7]:
df.head(-5)

,SPY,BND
Date,,
2007-04-10 04:00:00+00:00,106.189384,46.617035
2007-04-11 04:00:00+00:00,105.756165,46.493122
2007-04-12 04:00:00+00:00,106.226074,46.486931
2007-04-13 04:00:00+00:00,106.710747,46.412579
2007-04-16 04:00:00+00:00,107.724068,46.455948
...,...,...
2022-12-20 05:00:00+00:00,380.540009,72.598373
2022-12-21 05:00:00+00:00,386.230011,72.767975
2022-12-22 05:00:00+00:00,380.720001,72.758003


In [8]:
weather_hist.head()

,apparent_temperature (°F),precipitation (mm),cloudcover (%)
time,,,
2010-01-01 00:00:00,26.8,0.0,99
2010-01-01 01:00:00,27.3,0.0,91
2010-01-01 02:00:00,27.0,0.2,89
2010-01-01 03:00:00,27.1,0.4,91
2010-01-01 04:00:00,26.9,0.5,90


In [9]:
# remove time zone from df index
df.index = pd.Series(df.index).dt.tz_localize(None)

In [10]:
# join weather_hist with df on date
df = df.join(weather_hist, how='inner',on='Date', lsuffix='_df', rsuffix='_weather_hist')

In [11]:
# rename apparent_temperature to temp
df.rename(columns={
  'apparent_temperature (°F)':'temp',
  'precipitation (mm)':'precip',
  'cloudcover (%)':'clouds',
  }, inplace=True)

In [12]:
def weather_score(row):
    #! What if it is 72 -> big nm 
    # temp = min(abs(row['temp'] - 72.0001) ** -1, 50)
    temp = stats.norm.pdf(row['temp'], 72, 12.5) * 1500
    precip = -15 * row['precip'] ** 0.25
    return 0.5 * (100 - row['clouds']) + temp + precip
# how to optimize weather score? optimize for what nd when?
# generate column from lambda function over rows
df['niceness'] = df.apply(lambda row: weather_score(row), axis=1)
# df['niceness'] = df.apply(lambda row: weather_score(row['temp'], row['clouds'], row['precip']), axis=1)

In [13]:
# generate new column from lambda function over rows based on previous row
df['delta_nice'] = df.niceness.diff()

In [14]:
df.head()

,SPY,BND,temp,precip,clouds,niceness,delta_nice
Date,,,,,,,
2010-01-04 05:00:00,88.454201,54.839836,10.5,0.0,27,36.500265,NaN
2010-01-05 05:00:00,88.688354,55.000134,12.9,0.0,3,48.500670,12.000405
2010-01-06 05:00:00,88.750786,54.979218,15.6,0.0,36,32.001817,-16.498853
2010-01-07 05:00:00,89.125427,54.937405,18.6,0.0,4,48.005214,16.003397
2010-01-08 05:00:00,89.421997,54.993134,22.8,0.0,100,0.020703,-47.984511


In [17]:
# add buy column with true if delta_nice > 0
df['sell'] = df.delta_nice > 38
df['buy'] = df.delta_nice < 4
# df = df[df.index.year == 2016]
# create portfolio and run backtest
portfolio = vbt.Portfolio.from_signals(df.BND, df.buy, df.sell, freq='D', )
portfolio.stats()

Start                               2010-01-04 05:00:00
End                                 2019-12-31 05:00:00
Period                               2516 days 00:00:00
Start Value                                       100.0
End Value                                    132.543427
Total Return [%]                              32.543427
Benchmark Return [%]                          42.624375
Max Gross Exposure [%]                            100.0
Total Fees Paid                                     0.0
Max Drawdown [%]                               4.496162
Max Drawdown Duration                 352 days 00:00:00
Total Trades                                        289
Total Closed Trades                                 288
Total Open Trades                                     1
Open Trade PnL                                 0.200707
Win Rate [%]                                  55.555556
Best Trade [%]                                 2.361969
Worst Trade [%]                               -2

In [18]:
portfolio.beta()

0.8503177825624678

In [19]:
portfolio.plot().show()

In [24]:
df.head(-5)

,SPY,BND,temp,precip,clouds,niceness,delta_nice,sell,buy
Date,,,,,,,,,
2010-01-04 05:00:00,88.454201,54.839836,10.5,0.0,27,36.500265,NaN,False,False
2010-01-05 05:00:00,88.688354,55.000134,12.9,0.0,3,48.500670,12.000405,False,False
2010-01-06 05:00:00,88.750786,54.979218,15.6,0.0,36,32.001817,-16.498853,False,True
2010-01-07 05:00:00,89.125427,54.937405,18.6,0.0,4,48.005214,16.003397,False,False
2010-01-08 05:00:00,89.421997,54.993134,22.8,0.0,100,0.020703,-47.984511,False,True
...,...,...,...,...,...,...,...,...,...
2019-12-17 05:00:00,303.152039,78.143074,26.0,1.5,100,-16.545351,-24.174312,False,True
2019-12-18 05:00:00,303.170959,78.040726,25.8,0.0,28,36.051733,52.597084,True,False
2019-12-19 05:00:00,304.413635,78.096535,10.2,0.0,76,12.000236,-24.051498,False,True


In [25]:
returns_by_sector = {}
baseline_returns = {
    # 'SPY': 0.6036, 
    'SPY': 1.8926, 
    'XLK': 1.1167,
    'XLV': 0.5066, 
    'XLF': 0.6214,
    'XLP': 0.2703,
    'XLI': 0.5382,
    'XLE': -0.1008,
    'XLU': 0.4613,
    'XLRE': 0.2603,
    'XLY': 0.5988,
    'XLB': 0.3828,
    'BND': -0.0484,
}

for sector in sectors:
    portfolio = vbt.Portfolio.from_signals(df[sector], df.buy, df.sell, freq='D')
    # excess returns
    returns_by_sector[sector] = portfolio.total_return() - baseline_returns[sector]
    # get total perent change of sector

In [26]:
# create a bar chart from returns_by_sector
fig = go.Figure(data=[
    go.Bar(x=list(returns_by_sector.keys()), 
    y=list(returns_by_sector.values()))
    ])
fig.show()

In [29]:
# df = df[df.index.year]
num = 10
entry_points = np.linspace(-40,-1, num=num)
exit_points = np.linspace(0,40, num=num)
returns_mtx = pd.DataFrame(columns=entry_points, index=exit_points)
for entry in entry_points:
  for exit in exit_points:
    df['buy'] = df.delta_nice > entry
    df['sell'] = df.delta_nice < exit
    portfolio = vbt.Portfolio.from_signals(df.XLE, df.buy, df.sell, freq='D')
    returns_mtx[entry][exit] = portfolio.total_return()
returns_mtx.vbt.heatmap(
        xaxis_title = "entry",
        yaxis_title="exit").show()